In [ ]:
import pandas as pd
import numpy as np

from scipy.sparse import kronsum

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, text
import networkx as nx

## Generic functions

In [ ]:
def exp_kernel(train, sigma):
    """
    Computes the exponential kernel matrix for the given data and sigma.
    
    Parameters:
    train (np.array or pd.DataFrame): The input data matrix.
    sigma (float): The kernel bandwidth parameter.
    
    Returns:
    pd.DataFrame: The computed kernel matrix, rounded to 6 decimal places.
    """
    matrix_train = np.exp(-(train**2) / (2 * (sigma**2)))

    x = pd.DataFrame(matrix_train)
    x = np.round(x, 6)

    eigenvalues = np.linalg.eig(x)
    matrix = np.array(x)
    if not (np.sum(np.abs(eigenvalues[0]) > 0) == x.shape[0]) and (np.array_equal(matrix, matrix.T)):
        print("==============================")
        print("DOES NOT satisfy kernel condition")
        print("==============================")

    return x


### Common Parameters

In [ ]:
norm = "robustNorm"
numberOfTimeSteps = 14
debug_plot_figures = True


folders = ["s1", "s2", "s3", "s4"]

keys = ['AMG', 'ATF', 'CAR', 'CF1', 'CF2', 'CF3',
       'CF4', 'Falta', 'GCC', 'GLI', 'LIN', 'LIP', 'MAC', 'MON', 'NTI', 'OTR',
       'OXA', 'PAP', 'PEN', 'POL', 'QUI', 'SUL', 'TTC', 'pc_acinet',
       'pc_enterob', 'pc_enteroc', 'pc_pseud', 'pc_staph', 'pc_stenot',
       'pc_no_germ', 'isVM', 'numberOfPatients', 'numberOfPatientsMR',
       'neighbor_AMG', 'neighbor_ATF', 'neighbor_CAR', 'neighbor_CF1',
       'neighbor_CF2', 'neighbor_CF3', 'neighbor_CF4', 'neighbor_Falta',
       'neighbor_GCC', 'neighbor_GLI', 'neighbor_LIN', 'neighbor_LIP',
       'neighbor_MAC', 'neighbor_MON', 'neighbor_NTI', 'neighbor_OTR',
       'neighbor_OXA', 'neighbor_PAP', 'neighbor_PEN', 'neighbor_POL',
       'neighbor_QUI', 'neighbor_SUL', 'neighbor_TTC']

binary = ['AMG', 'ATF', 'CAR', 'CF1', 'CF2', 'CF3',
       'CF4', 'Falta', 'GCC', 'GLI', 'LIN', 'LIP', 'MAC', 'MON', 'NTI', 'OTR',
       'OXA', 'PAP', 'PEN', 'POL', 'QUI', 'SUL', 'TTC', 'pc_acinet',
       'pc_enterob', 'pc_enteroc', 'pc_pseud', 'pc_staph', 'pc_stenot',
       'pc_no_germ', 'isVM']

continues =  [variable for variable in keys if variable not in binary]

### Threshold value

In [ ]:
# Based on the threshold value, you can choose between: 0.6, 0.725, 0.85 and 0.975
threshold_val_init = 0.975
th_folder = "th_0975"
save_results = False

# 1.DTW

In [ ]:
buildGraph = "dtw"
numberOfFeatures = 56

### 1. Graph Constructed as tr[expK(DTW)]. Same Graph for Each Time Step
#### A Single Graph for all MTS

In [ ]:
dicc_thresholds = {}
for c in range(len(folders)):
    print("====> Folder:" + str(folders[c]) + " <====")
    dtw = pd.read_csv("../step1_graphEstimation/estimatedGraphs/"+buildGraph+"/"+folders[c]+"/X_train_allMTS.csv")
    K = exp_kernel(dtw, 1.5)
    K = K - np.eye(K.shape[0])
    
    edges_bef = np.count_nonzero(K)
    print("Number of non-zero values before applying the threshold:", edges_bef)
    s = K.copy()
    min_value = s.min().min()
    max_value = s.max().max()
    s = (s - min_value) / (max_value - min_value)
    s[np.abs(s) < threshold_val_init] = 0
    edges_aft = np.count_nonzero(s)
    print("Number of non-zero values after applying the threshold:", edges_aft)
    print("%:", (edges_aft * 100) / (numberOfFeatures * numberOfFeatures))
    print(s.shape)
    
    pd.DataFrame(s).to_csv("./dtw/"+folders[c]+"/graph_Xtr_th_"+str(threshold_val_init)+".csv", index=False)

### 2. Graph Constructed as tr[expK(DTW)]. Same Graph for Each Time Step
#### A Single Graph for static data

In [ ]:
dicc_thresholds = {}
for c in range(len(folders)):
    print("====> Folder:" + str(folders[c]) + " <====")
    dtw = pd.read_csv("../step1_graphEstimation/estimatedGraphs/"+buildGraph+"/"+folders[c]+"/X_train_STATIC.csv")
    K = exp_kernel(dtw, 1.5)
    K = K - np.eye(K.shape[0])
    
    edges_bef = np.count_nonzero(K)
    print("Number of non-zero values before applying the threshold:", edges_bef)
    s = K.copy()
    min_value = s.min().min()
    max_value = s.max().max()
    s = (s - min_value) / (max_value - min_value)
    s[np.abs(s) < threshold_val_init] = 0
    edges_aft = np.count_nonzero(s)
    print("Number of non-zero values after applying the threshold:", edges_aft)
    print("%:", (edges_aft * 100) / (numberOfFeatures * numberOfFeatures))
    print(s.shape)
    
    pd.DataFrame(s).to_csv("./dtw/"+folders[c]+"/static_graph_Xtr_th_"+str(threshold_val_init)+".csv", index=False)